In [ ]:
import pandas as pd
import plotly.graph_objects as go

def bollinger_bands(data, column='BASKET_MINUS_SYNTHETIC', window=20, num_std=2):
    """
    Calculate Bollinger Bands for a given DataFrame and column.
    
    Parameters:
    data (pd.DataFrame): The input DataFrame.
    column (str): The name of the column to calculate Bollinger Bands for.
    window (int): The rolling window size for the Simple Moving Average (SMA).
    num_std (int): The number of standard deviations for the upper and lower bands.
    
    Returns:
    pd.DataFrame: The input DataFrame with additional columns for the Bollinger Bands.
    """
    # Calculate the Simple Moving Average (SMA)
    data['SMA'] = data[column].rolling(window=window).mean()
    
    # Calculate the standard deviation
    data['STD'] = data[column].rolling(window=window).std()
    
    # Calculate the upper and lower Bollinger Bands
    data['UPPER_BAND'] = data['SMA'] + (data['STD'] * num_std)
    data['LOWER_BAND'] = data['SMA'] - (data['STD'] * num_std)
    
    return data

# Example usage
df_fairs = bollinger_bands(df_fairs, window=200, num_std=1.75)

# Create the plot
fig = go.Figure()

# Add the BASKET_MINUS_SYNTHETIC line
fig.add_trace(go.Scatter(x=df_fairs['timestamp'], y=df_fairs['BASKET_MINUS_SYNTHETIC'],
                         mode='lines', name='BASKET_MINUS_SYNTHETIC'))

# Add the Upper Bollinger Band line
fig.add_trace(go.Scatter(x=df_fairs['timestamp'], y=df_fairs['UPPER_BAND'],
                         mode='lines', name='Upper Band', line=dict(color='red')))

# Add the Simple Moving Average (SMA) line
fig.add_trace(go.Scatter(x=df_fairs['timestamp'], y=df_fairs['SMA'],
                         mode='lines', name='SMA', line=dict(color='blue')))

# Add the Lower Bollinger Band line
fig.add_trace(go.Scatter(x=df_fairs['timestamp'], y=df_fairs['LOWER_BAND'],
                         mode='lines', name='Lower Band', line=dict(color='green')))

# Update the layout
fig.update_layout(title='Bollinger Bands',
                  xaxis_title='Timestamp',
                  yaxis_title='BASKET_MINUS_SYNTHETIC')

# Display the plot
fig.show()

In [ ]:
from tqdm import tqdm
def identify_regime(data, mean=370, lookback=20):
    """
    Identify the regime based on the price's interaction with Bollinger Bands.
    
    Parameters:
    data (pd.DataFrame): The input DataFrame containing Bollinger Bands.
    lookback (int): The lookback period to determine the regime.
    
    Returns:
    pd.Series: A series containing the identified regime for each row.
    """
    regimes = []
    
    for i in tqdm(range(len(data))):
        if i < lookback:
            regimes.append('NEUTRAL')
        else:
            period = data.iloc[i-lookback+1:i+1]
            price = period['BASKET_MINUS_SYNTHETIC']
            upper = period['UPPER_BAND']
            lower = period['LOWER_BAND']
            
            # Count contiguous touches of upper and lower bands
            touched_upper = 0
            touched_lower = 0
            touching_upper = False
            touching_lower = False
            
            for j in range(len(period)):
                if price.iloc[j] >= upper.iloc[j]:
                    if not touching_upper:
                        touched_upper += 1
                        touching_upper = True
                    touching_lower = False
                elif price.iloc[j] <= lower.iloc[j]:
                    if not touching_lower:
                        touched_lower += 1
                        touching_lower = True
                    touching_upper = False
                else:
                    touching_upper = False
                    touching_lower = False
            
            if touched_upper > 1 and touched_lower > 1:
                regimes.append('OSCILLATING')
            elif touched_upper > 1:
                regimes.append('RIDING_UPPER')
            elif touched_lower > 1:
                regimes.append('RIDING_LOWER')
            else:
                regimes.append('NEUTRAL')
    
    return pd.Series(regimes, index=data.index)
# Apply the identify_regime function to your DataFrame
df_fairs['REGIME'] = identify_regime(df_fairs, lookback=300)

In [ ]:
def generate_signals(data, mean_price):
    """
    Generate trading signals based on the identified regimes and Bollinger Bands.
    
    Parameters:
    data (pd.DataFrame): The input DataFrame containing Bollinger Bands and regimes.
    mean_price (float): The mean price to compare against when entering riding regimes.
    
    Returns:
    pd.Series: A series containing the generated trading signals for each row.
    """
    price = data['BASKET_MINUS_SYNTHETIC']
    upper = data['UPPER_BAND']
    lower = data['LOWER_BAND']
    regime = data['REGIME']
    
    signals = pd.Series(index=data.index, dtype=object)
    
    for i in tqdm(range(1, len(data))):
        if regime[i] == 'OSCILLATING':
            if price[i] >= upper[i] and price[i-1] < upper[i-1]:
                signals[i] = 'SHORT'
            elif price[i] >= upper[i] and price[i+1] < upper[i+1]:
                signals[i] = "SHORT"
            elif price[i] <= lower[i] and price[i-1] > lower[i-1]:
                signals[i] = 'LONG'
            elif price[i] <= lower[i] and price[i+1] > lower[i+1]:
                signals[i] = 'LONG'
            
        elif regime[i] == 'NEUTRAL' and regime[i-1] != 'NEUTRAL':
            signals[i] = 'CLEAR'
        elif regime[i] == 'RIDING_UPPER' and regime[i-1] != 'RIDING_UPPER':
            if price[i] > mean_price:
                signals[i] = 'CLEAR'
            else:
                signals[i] = 'LONG'
        elif regime[i] == 'RIDING_LOWER' and regime[i-1] != 'RIDING_LOWER':
            if price[i] < mean_price:
                signals[i] = 'CLEAR'
            else:
                signals[i] = 'SHORT'
    
    return signals

mean_price = df_fairs['BASKET_MINUS_SYNTHETIC'].mean()
df_fairs['SIGNAL'] = generate_signals(df_fairs, mean_price)

In [ ]:
def update_position(signals):
    """
    Update the position based on the last non-nan signal.
    
    Parameters:
    signals (pd.Series): The series containing the trading signals.
    
    Returns:
    pd.Series: A series containing the updated positions.
    """
    last_signal = None
    positions = []
    
    for signal in signals:
        if pd.notna(signal):
            last_signal = signal
        
        if last_signal == 'LONG':
            position = 60
        elif last_signal == 'SHORT':
            position = -60
        else:
            position = 0
        
        positions.append(position)
    
    return pd.Series(positions, index=signals.index)

# Update the position based on the trading signals
df_fairs['POSITION'] = update_position(df_fairs['SIGNAL'])

In [ ]:
# Initialize the "CASH" column with zeros
df_fairs['CASH'] = 0

# Calculate the position differences
df_fairs['POSITION_DIFF'] = df_fairs['POSITION'].diff().fillna(0)

# Update the "CASH" column based on position changes
df_fairs.loc[position_diff != 0, 'CASH'] = -df_fairs['POSITION_DIFF'] * df_fairs['BASKET_MINUS_SYNTHETIC']

# Calculate the cumulative cash
df_fairs['CUMULATIVE_CASH'] = df_fairs['CASH'].cumsum()

df_fairs['PNL'] = df_fairs['CUMULATIVE_CASH'] + df_fairs['POSITION'] * df_fairs["BASKET_MINUS_SYNTHETIC"]

In [ ]:
'''
backtestin code
'''

In [ ]:
from tqdm import tqdm
spread['std30'] = spread['spread'].rolling(window=30).std()
z_score = (spread['spread'].to_numpy() - 376) / spread['std30'].to_numpy()
spread_market = pd.DataFrame({'timestamp': df_synthetic['timestamp'].to_numpy(), 'swmid': df_gift_basket['swmid'].to_numpy() - df_synthetic['swmid'].to_numpy()})

def cross_spread(cash, quantity):
    return cash - abs(quantity) * 10

def backtest(thresh, target_position, std_window, sma_window, verbose=False):
    cash = 0
    position = 0
    pnl_hist = []
    position_hist = []
    cash_hist = []
    spread[f'std{std_window}'] = spread['spread'].rolling(window=std_window).std()
    spread[f'sma{sma_window}'] = spread['spread'].rolling(window=sma_window).mean()
    z_score = (spread['spread'].to_numpy() - spread[f'sma{sma_window}']) / spread[f'std{std_window}'].to_numpy()
    spread_market['spread_z'] = z_score
    for index, row in spread_market.iterrows():
        if index == 0:
            continue
        swmid = row['swmid']
        
        if row['spread_z'] > thresh and position != -target_position:
        
            
            quantity = -target_position - position
            cash -= (-target_position - position) * swmid
            cash = cross_spread(cash, quantity)
            position = -target_position
            
            if verbose:
                print(f"SELL {quantity} AT PRICE {swmid} AT TIME {row['timestamp']}")
        
        if row['spread_z'] < -thresh and position != target_position:
            quantity = target_position - position
            cash -= (target_position - position) * swmid
            cash = cross_spread(cash, quantity)
            position = target_position
            
            if verbose:
                print(f"BUY {quantity} FOR PRICE {swmid} AT TIME {row['timestamp']}")
    
        position_hist.append(position)
        cash_hist.append(cash)
        pnl_hist.append(cash + position * swmid)
        
    if verbose:
        print(f"PNL: {pnl_hist[-1]}")
        
    return pnl_hist

In [ ]:
from tqdm import tqdm
spread['std30'] = spread['spread'].rolling(window=30).std()
z_score = (spread['spread'].to_numpy() - 376) / spread['std30'].to_numpy()
spread_market = pd.DataFrame({'timestamp': df_synthetic['timestamp'].to_numpy(), 'swmid': df_gift_basket['swmid'].to_numpy() - df_synthetic['swmid'].to_numpy()})

def cross_spread(cash, quantity):
    return cash - abs(quantity) * 10

def backtest(take_thresh,clear_thresh, target_position, std_window, verbose=False):
    cash = 0
    position = 0
    pnl_hist = []
    position_hist = []
    cash_hist = []
    spread[f'std{std_window}'] = spread['spread'].rolling(window=std_window).std()
    z_score = (spread['spread'].to_numpy() - 376) / spread[f'std{std_window}'].to_numpy()
    spread_market['spread_z'] = z_score
    for index, row in spread_market.iterrows():
        if index == 0:
            continue
        swmid = row['swmid']
        
        if row['spread_z'] > take_thresh and position != -target_position:
            quantity = -target_position - position
            cash -= (-target_position - position) * swmid
            cash = cross_spread(cash, quantity)
            position = -target_position
            
            if verbose:
                print(f"SELL {quantity} AT PRICE {swmid} AT TIME {row['timestamp']}")
        
        if row['spread_z'] < -take_thresh and position != target_position:
            quantity = target_position - position
            cash -= (target_position - position) * swmid
            cash = cross_spread(cash, quantity)
            position = target_position
            
            if verbose:
                print(f"BUY {quantity} FOR PRICE {swmid} AT TIME {row['timestamp']}")
            
        if (row['spread_z'] < clear_thresh and row['spread_z'] > -clear_thresh) and position != 0:
            quantity = -position
            cash -= (quantity * swmid)
            cash = cross_spread(cash, quantity)
            position = 0
            
            if verbose:
                print(f"CLEAR {quantity} FOR PRICE {swmid} AT TIME {row['timestamp']}")
    
        position_hist.append(position)
        cash_hist.append(cash)
        pnl_hist.append(cash + position * swmid)
    if verbose:
        print(f"PNL: {pnl_hist[-1]}")
    return pnl_hist

In [ ]:
position_opt = [60]
thresh_opt = [1,2,3,5,6,7,7.5,8,9,10,15,20,25]
std_window_opt = [10,20,25,30,35,40,50]
sma_window_opt = [10,20,25,30,35,40,50,75, 100, 125, 150, 200, 300, 500]
opt = []
for thresh in tqdm(thresh_opt): 
    for std_window in std_window_opt: 
        for sma_window in sma_window_opt:
            for position in position_opt:
                pnl = backtest(thresh, position, std_window, sma_window)
                opt.append({"thresh": thresh, "position": position, "std_window": std_window, "sma_window": sma_window, "pnl": pnl})
#                 print("="*80)
#                 print(f"Thresh: {thresh}, Position: {position}, Std Window: {std_window}, PnL: {pnl[-1]}")
#                 print("="*80)

In [ ]:
import pandas as pd

# Set the threshold values
upper_threshold = 0.006  # Threshold for selling option
lower_threshold = -0.006  # Threshold for buying option
close_threshold = 0.0001  # Threshold for clearing position

# Initialize variables
position = 0
pnl = 0
vega_pnl = 0
trade_history = []

# Iterate over each row in the dataframe
for idx, row in df_backtest.iterrows():
    implied_vol = row['implied_vol']
    if idx == 0:
        continue
    prev_implied_vol = df_backtest.iloc[idx-1]['implied_vol']
    mid_price_coupon = row['mid_price_coupon']
    mid_price_coconut = row['mid_price_coconut']
    vega = row['vega']
    d = row['delta']

    # Check if implied vol is above the upper threshold and no current position
    if implied_vol > implied_vol_mean + upper_threshold and position == 0:
        # Sell 1 delta hedged option
        position = -1
        entry_price_coupon = mid_price_coupon
        entry_price_coconut = mid_price_coconut
        trade_history.append((-1, entry_price_coupon, entry_price_coconut, implied_vol))

    # Check if implied vol is below the lower threshold and no current position
    elif implied_vol < implied_vol_mean + lower_threshold and position == 0:
        # Buy 1 delta hedged option
        position = 1
        entry_price_coupon = mid_price_coupon
        entry_price_coconut = mid_price_coconut
        trade_history.append((1, entry_price_coupon, entry_price_coconut, implied_vol))

    # Check if implied vol is within the close threshold and there is a current position
    elif abs(implied_vol - implied_vol_mean) <= close_threshold and position != 0:
        # Clear the position
        pnl += position * (mid_price_coupon - entry_price_coupon + d * (entry_price_coconut - mid_price_coconut))
        position = 0
        trade_history.append((0, mid_price_coupon, mid_price_coconut, implied_vol))

    if position != 0:
        vega_pnl += position * vega * (implied_vol - prev_implied_vol) * 100
# Calculate final PnL if there is still an open position
if position != 0:
    pnl += position * (mid_price_coupon - entry_price_coupon + d * (entry_price_coconut - mid_price_coconut))

# Print the trade history and final PnL
print("Trade History:")
for trade in trade_history:
    print(f"Position: {trade[0]}, Option Price: {trade[1]}, Underlying Price: {trade[2]}, Implied Volatility: {trade[3]}")

print(f"\nFinal PnL: {pnl}")

In [ ]:
import pandas as pd

# Set the threshold values
upper_threshold = 0.005  # Threshold for selling option
lower_threshold = -0.005  # Threshold for buying option

# Initialize variables
position = 0
pnl = 0
trade_history = []

# Iterate over each row in the dataframe
for _, row in df_backtest.iterrows():
    implied_vol = row['implied_vol']
    mid_price_coupon = row['mid_price_coupon']
    mid_price_coconut = row['mid_price_coconut']
    d = row['delta']

    # Check if implied vol is above the upper threshold
    if implied_vol > implied_vol_mean + upper_threshold:
        # Sell to target position of -1
        if position > -1:
            quantity = -1 - position
            position = -1
            entry_price_coupon = mid_price_coupon
            entry_price_coconut = mid_price_coconut
            trade_history.append((quantity, entry_price_coupon, entry_price_coconut, implied_vol))

    # Check if implied vol is below the lower threshold
    elif implied_vol < implied_vol_mean + lower_threshold:
        # Buy to target position of 1
        if position < 1:
            quantity = 1 - position
            position = 1
            entry_price_coupon = mid_price_coupon
            entry_price_coconut = mid_price_coconut
            trade_history.append((quantity, entry_price_coupon, entry_price_coconut, implied_vol))

# Calculate final PnL for the remaining position
if position != 0:
    pnl += position * (mid_price_coupon - entry_price_coupon + d * (entry_price_coconut - mid_price_coconut))

# Print the trade history and final PnL
print("Trade History:")
for trade in trade_history:
    print(f"Quantity: {trade[0]}, Option Price: {trade[1]}, Underlying Price: {trade[2]}, Implied Volatility: {trade[3]}")

print(f"\nFinal PnL: {pnl}")